In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import statsmodels.api as smf

pd.set_option('display.max_columns', 500)


%matplotlib inline

In [2]:
#removed the stops that are outside of San Francisco, Need to add this step to the linking buffer data step
estimate = pd.read_csv('E:/Transit-Casa-Alex/Output/Modeling/2009/Tenth_Data_rs_day.csv',thousands = ',')
estimate = estimate.drop(estimate[estimate['STOP_ID'].isin([4341,5585,5588,5594,7099,7554,7606,7842,4808,4809,4895,6283,6384,6385,6388,6389,7522,7127])].index,axis = 0)

In [3]:
estimate['FREQ_S'] = 1 / (estimate['HEADWAY_S'] / 60.0)

In [4]:
print(max(estimate.RUNSPEED))
print(min(estimate.RUNSPEED))

909.32
0.0


In [6]:
print(max(estimate.TOTSPEED))
print(min(estimate.TOTSPEED))

909.32
0.0


In [7]:
estimate.RUNSPEED = estimate.RUNSPEED.clip(upper = 40)
estimate.RUNSPEED = estimate.TOTSPEED.clip(upper = 40)


In [8]:
#leave out travel time to work for now
#estimate['Workers_16_And_Over_Mean_Travel_Time_To_Work_(Minutes)'] = estimate['Workers_16_And_Over_Mean_Travel_Time_To_Work_(Minutes)'].astype(float)

In [122]:
estimate.info(max_cols = 99999)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 17995 entries, 0 to 18092
Data columns (total 122 columns):
Unnamed: 0                                                17995 non-null int64
MONTH                                                     17995 non-null object
DOW                                                       17995 non-null int64
AGENCY_ID                                                 17995 non-null object
ROUTE_SHORT_NAME                                          17995 non-null object
DIR                                                       17995 non-null int64
SEQ                                                       17995 non-null int64
NUMDAYS                                                   17995 non-null int64
OBSDAYS                                                   17995 non-null float64
TRIP_STOPS                                                17995 non-null int64
OBS_TRIP_STOPS                                            17995 non-null int64
IMP_TRIP_STOPS  

In [188]:
estimate.head()

,Unnamed: 0,MONTH,DOW,AGENCY_ID,ROUTE_SHORT_NAME,DIR,SEQ,NUMDAYS,OBSDAYS,TRIP_STOPS,OBS_TRIP_STOPS,IMP_TRIP_STOPS,WGT_TRIP_STOPS,STOP_ID,ROUTE_LONG_NAME,ROUTE_TYPE,TRIP_HEADSIGN,HEADWAY_S,FARE,STOPNAME,STOPNAME_AVL,STOP_LAT_x,STOP_LON_x,EOL,SOL,TIMEPOINT,ARRIVAL_TIME_DEV,DEPARTURE_TIME_DEV,DWELL_S,DWELL,RUNTIME_S,RUNTIME,TOTTIME_S,TOTTIME,SERVMILES_S,SERVMILES,RUNSPEED_S,RUNSPEED,TOTSPEED_S,TOTSPEED,ONTIME5,ON,OFF,LOAD_ARR,LOAD_DEP,PASSMILES,PASSHOURS,WAITHOURS,FULLFARE_REV,PASSDELAY_DEP,PASSDELAY_ARR,RDBRDNGS,DOORCYCLES,WHEELCHAIR,BIKERACK,CAPACITY,VC,CROWDED,CROWDHOURS,STOP_LAT_y,STOP_LON_y,geometry,index_right,ALAND00,AWATER00,COUNTYFP00,CTIDFP00,NAME00,TRACTCE00,Id,Id2,Geography,Total_Pop,Total_Housing_Units,Workers_16_And_Over_Mean_Travel_Time_To_Work_(Minutes),VAC_RATE,OCC_RATE,PER_INCOME_0-15,PER_INCOME_15-50,PER_INCOME_50-100,PER_INCOME_100+,AVG_HH_SIZE,PER_HH_0VEH,POP_DEN_ACS,HH_DEN_ACS,HU_DEN_ACS,geometry_a,Average Hourly Price,parking_rate,PRICE1HR,PRICE2HR,PRICE3HR,PRICE4HR,PRICE12HR,PRICEDAY,PRICEDAYDISCOUNT,BART_FROMS,BART_TOS,CALTRAIN_OFF,CALTRAIN_ON,MUNI_RAIL_ALIGHTINGS,MUNI_RAIL_BOARDINGS,EDHLTH_RAC_SCALED,EDHLTH_WAC_SCALED,EMP_RAC_SCALED,EMP_WAC_SCALED,LEISER_RAC_SCALED,LEISER_WAC_SCALED,OTHER_RAC_SCALED,OTHER_WAC_SCALED,RETAIL_RAC_SCALED,RETAIL_WAC_SCALED,HOUSING_09_SCALED,NUM_BUS_STOPS,Count_,FID_1,STOP_LAT,STOP_LON,geometry_e,EDD_EMP,geometry_f,FREQ_S
0,0,2009-10-01,1,SFMTA,016AX,0,3,22,18.0,13,2,0.0,13.0,3612,NORIEGA A EXP,3,48TH AVENUE,10.666667,2.0,CYRIL MAGNIN ST & MARKET ST,5TH ST NORTH&MARKET ST NW,37.784011,-122.408210,0,1,1.0,-3.875743,-0.070542,0.0,49.476280,0.00,0.000000,0.00,49.476280,0.000,0.000,0.00,0.00,0.00,0.00,0.737434,258.292460,0.240741,0.240741,258.292460,0.000000,20.102898,23.016448,516.584921,287.502463,91.157255,0.0,21.049339,0.000000,0.0,189.0,0.000294,0.0,0.0,37.784011,-122.408210,POINT (-122.40821 37.784011),150.0,198169.0,0.0,75.0,6.075012e+09,125.0,12500.0,1400000US06075012500,6.075012e+09,"Census Tract 125, San Francisco County, Califo...",6317.0,4775.0,29.4,0.164817,0.835183,0.650953,0.278084,0.067202,0.003761,1.584002,0.912738,129.001163,81.440025,97.511565,POINT (6010258.215635454 2113559.86915681),350.0,0.320246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4673.0,4772.0,NaN,NaN,2496.0,4136.0,39.308818,199.634504,232.201161,4068.828658,32.287304,963.525560,144.291088,1437.649130,16.313950,1468.019464,717.267880,10.0,0.0,535.0,37.784011,-122.408210,POLYGON ((-122.4063833032154 37.78404055077128...,7134.0,POLYGON ((-122.4063833032154 37.78404055077128...,5.625
1,1,2009-10-01,1,SFMTA,016AX,0,4,22,0.0,13,0,0.0,13.0,7305,NORIEGA A EXP,3,48TH AVENUE,10.666667,2.0,EDDY ST & CYRIL MAGNIN ST,NaN,37.784386,-122.408599,0,0,NaN,NaN,NaN,0.0,NaN,2.99,NaN,2.99,NaN,0.429,0.429,8.61,NaN,8.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.784386,-122.408599,POINT (-122.408599 37.784386),150.0,198169.0,0.0,75.0,6.075012e+09,125.0,12500.0,1400000US06075012500,6.075012e+09,"Census Tract 125, San Francisco County, Califo...",6317.0,4775.0,29.4,0.164817,0.835183,0.650953,0.278084,0.067202,0.003761,1.584002,0.912738,129.001163,81.440025,97.511565,POINT (6010148.60787045 2113698.678689688),350.0,0.320246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2496.0,4136.0,53.874574,180.151219,306.527823,3867.629831,43.926365,1155.305396,184.122863,1410.526463,24.604021,1121.646752,980.850696,12.0,0.0,3502.0,37.784386,-122.408599,POLYGON ((-122.4067722941473 37.78441555679537...,7852.0,POLYGON ((-122.4067722941473 37.78441555679537...,5.625
2,2,2009-10-01,1,SFMTA,016AX,0,5,22,18.0,13,2,0.0,13.0,6733,NORIEGA A EXP,3,48TH AVENUE,10.666667,2.0,TURK ST & TAYLOR ST,TURK ST&TAYLOR ST N-MB/BZ,37.783387,-122.409895,0,0,0.0,0.270194,0.350709,0.0,1.049097,11.05,18.514923,11.05,19.564020,1.287,1.287,6.99,3.95,6.99,3.95,0.728175,7.129365,0.240741,258.292460,265.181085,25.570954,6.700418,0.531593,14.258730,5.972606,5.569709,0.0,6.580820,0.361111,0.0,189.0,0.315375,0.0,0.0,37.783387,-122.409895,POINT (-122

In [124]:
estimate[['HEADWAY_S', 'FREQ_S']].head()

,HEADWAY_S,FREQ_S
0,10.666667,5.625
1,10.666667,5.625
2,10.666667,5.625
3,10.666667,5.625
4,10.666667,5.625


In [125]:
for column in estimate.columns:
        print(column)

Unnamed: 0
MONTH
DOW
AGENCY_ID
ROUTE_SHORT_NAME
DIR
SEQ
NUMDAYS
OBSDAYS
TRIP_STOPS
OBS_TRIP_STOPS
IMP_TRIP_STOPS
WGT_TRIP_STOPS
STOP_ID
ROUTE_LONG_NAME
ROUTE_TYPE
TRIP_HEADSIGN
HEADWAY_S
FARE
STOPNAME
STOPNAME_AVL
STOP_LAT_x
STOP_LON_x
EOL
SOL
TIMEPOINT
ARRIVAL_TIME_DEV
DEPARTURE_TIME_DEV
DWELL_S
DWELL
RUNTIME_S
RUNTIME
TOTTIME_S
TOTTIME
SERVMILES_S
SERVMILES
RUNSPEED_S
RUNSPEED
TOTSPEED_S
TOTSPEED
ONTIME5
ON
OFF
LOAD_ARR
LOAD_DEP
PASSMILES
PASSHOURS
WAITHOURS
FULLFARE_REV
PASSDELAY_DEP
PASSDELAY_ARR
RDBRDNGS
DOORCYCLES
WHEELCHAIR
BIKERACK
CAPACITY
VC
CROWDED
CROWDHOURS
STOP_LAT_y
STOP_LON_y
geometry
index_right
ALAND00
AWATER00
COUNTYFP00
CTIDFP00
NAME00
TRACTCE00
Id
Id2
Geography
Total_Pop
Total_Housing_Units
Workers_16_And_Over_Mean_Travel_Time_To_Work_(Minutes)
VAC_RATE
OCC_RATE
PER_INCOME_0-15
PER_INCOME_15-50
PER_INCOME_50-100
PER_INCOME_100+
AVG_HH_SIZE
PER_HH_0VEH
POP_DEN_ACS
HH_DEN_ACS
HU_DEN_ACS
geometry_a
Average Hourly Price
parking_rate
PRICE1HR
PRICE2HR
PRICE3HR
PRICE4HR


In [9]:
drop = ['Unnamed: 0',
'STOP_LAT_x',
'STOP_LON_x',
'STOP_LAT_y',
'STOP_LON_y',
'geometry',
'index_right',
'ALAND00',
'AWATER00',
'COUNTYFP00',
'CTIDFP00',
'NAME00',
'TRACTCE00',
'Id',
'Id2',
'Geography',
'geometry_a',
'geometry_e',
'geometry_f',
'STOP_LAT',
'STOP_LON',
'FID_1']

In [10]:
estimate = estimate.drop(drop,axis = 1)

In [128]:
for column in estimate.columns:
    print(column)

MONTH
DOW
AGENCY_ID
ROUTE_SHORT_NAME
DIR
SEQ
NUMDAYS
OBSDAYS
TRIP_STOPS
OBS_TRIP_STOPS
IMP_TRIP_STOPS
WGT_TRIP_STOPS
STOP_ID
ROUTE_LONG_NAME
ROUTE_TYPE
TRIP_HEADSIGN
HEADWAY_S
FARE
STOPNAME
STOPNAME_AVL
EOL
SOL
TIMEPOINT
ARRIVAL_TIME_DEV
DEPARTURE_TIME_DEV
DWELL_S
DWELL
RUNTIME_S
RUNTIME
TOTTIME_S
TOTTIME
SERVMILES_S
SERVMILES
RUNSPEED_S
RUNSPEED
TOTSPEED_S
TOTSPEED
ONTIME5
ON
OFF
LOAD_ARR
LOAD_DEP
PASSMILES
PASSHOURS
WAITHOURS
FULLFARE_REV
PASSDELAY_DEP
PASSDELAY_ARR
RDBRDNGS
DOORCYCLES
WHEELCHAIR
BIKERACK
CAPACITY
VC
CROWDED
CROWDHOURS
Total_Pop
Total_Housing_Units
Workers_16_And_Over_Mean_Travel_Time_To_Work_(Minutes)
VAC_RATE
OCC_RATE
PER_INCOME_0-15
PER_INCOME_15-50
PER_INCOME_50-100
PER_INCOME_100+
AVG_HH_SIZE
PER_HH_0VEH
POP_DEN_ACS
HH_DEN_ACS
HU_DEN_ACS
Average Hourly Price
parking_rate
PRICE1HR
PRICE2HR
PRICE3HR
PRICE4HR
PRICE12HR
PRICEDAY
PRICEDAYDISCOUNT
BART_FROMS
BART_TOS
CALTRAIN_OFF
CALTRAIN_ON
MUNI_RAIL_ALIGHTINGS
MUNI_RAIL_BOARDINGS
EDHLTH_RAC_SCALED
EDHLTH_WAC_SCALED


In [11]:
change = ['EMP_RAC_SCALED','EMP_WAC_SCALED','LEISER_RAC_SCALED','LEISER_WAC_SCALED','OTHER_RAC_SCALED','OTHER_WAC_SCALED','RETAIL_RAC_SCALED','RETAIL_WAC_SCALED','BART_FROMS','BART_TOS','MUNI_RAIL_ALIGHTINGS','MUNI_RAIL_BOARDINGS','CALTRAIN_OFF','CALTRAIN_ON','NUM_BUS_STOPS']

In [12]:
for column in change:
    estimate[column] = estimate[column].fillna(0)

In [13]:
estimate = estimate.drop(estimate[np.isnan(estimate['ON'])].index,axis = 0)

In [194]:
estimate[np.isnan(estimate['ON'])]

,MONTH,DOW,AGENCY_ID,ROUTE_SHORT_NAME,DIR,SEQ,NUMDAYS,OBSDAYS,TRIP_STOPS,OBS_TRIP_STOPS,IMP_TRIP_STOPS,WGT_TRIP_STOPS,STOP_ID,ROUTE_LONG_NAME,ROUTE_TYPE,TRIP_HEADSIGN,HEADWAY_S,FARE,STOPNAME,STOPNAME_AVL,EOL,SOL,TIMEPOINT,ARRIVAL_TIME_DEV,DEPARTURE_TIME_DEV,DWELL_S,DWELL,RUNTIME_S,RUNTIME,TOTTIME_S,TOTTIME,SERVMILES_S,SERVMILES,RUNSPEED_S,RUNSPEED,TOTSPEED_S,TOTSPEED,ONTIME5,ON,OFF,LOAD_ARR,LOAD_DEP,PASSMILES,PASSHOURS,WAITHOURS,FULLFARE_REV,PASSDELAY_DEP,PASSDELAY_ARR,RDBRDNGS,DOORCYCLES,WHEELCHAIR,BIKERACK,CAPACITY,VC,CROWDED,CROWDHOURS,Total_Pop,Total_Housing_Units,Workers_16_And_Over_Mean_Travel_Time_To_Work_(Minutes),VAC_RATE,OCC_RATE,PER_INCOME_0-15,PER_INCOME_15-50,PER_INCOME_50-100,PER_INCOME_100+,AVG_HH_SIZE,PER_HH_0VEH,POP_DEN_ACS,HH_DEN_ACS,HU_DEN_ACS,Average Hourly Price,parking_rate,PRICE1HR,PRICE2HR,PRICE3HR,PRICE4HR,PRICE12HR,PRICEDAY,PRICEDAYDISCOUNT,BART_FROMS,BART_TOS,CALTRAIN_OFF,CALTRAIN_ON,MUNI_RAIL_ALIGHTINGS,MUNI_RAIL_BOARDINGS,EDHLTH_RAC_SCALED,EDHLTH_WAC_SCALED,EMP_RAC_SCALED,EMP_WAC_SCALED,LEISER_RAC_SCALED,LEISER_WAC_SCALED,OTHER_RAC_SCALED,OTHER_WAC_SCALED,RETAIL_RAC_SCALED,RETAIL_WAC_SCALED,HOUSING_09_SCALED,NUM_BUS_STOPS,Count_,EDD_EMP,FREQ_S


In [195]:
estimate[np.isnan(estimate['BART_FROMS'])]

,MONTH,DOW,AGENCY_ID,ROUTE_SHORT_NAME,DIR,SEQ,NUMDAYS,OBSDAYS,TRIP_STOPS,OBS_TRIP_STOPS,IMP_TRIP_STOPS,WGT_TRIP_STOPS,STOP_ID,ROUTE_LONG_NAME,ROUTE_TYPE,TRIP_HEADSIGN,HEADWAY_S,FARE,STOPNAME,STOPNAME_AVL,EOL,SOL,TIMEPOINT,ARRIVAL_TIME_DEV,DEPARTURE_TIME_DEV,DWELL_S,DWELL,RUNTIME_S,RUNTIME,TOTTIME_S,TOTTIME,SERVMILES_S,SERVMILES,RUNSPEED_S,RUNSPEED,TOTSPEED_S,TOTSPEED,ONTIME5,ON,OFF,LOAD_ARR,LOAD_DEP,PASSMILES,PASSHOURS,WAITHOURS,FULLFARE_REV,PASSDELAY_DEP,PASSDELAY_ARR,RDBRDNGS,DOORCYCLES,WHEELCHAIR,BIKERACK,CAPACITY,VC,CROWDED,CROWDHOURS,Total_Pop,Total_Housing_Units,Workers_16_And_Over_Mean_Travel_Time_To_Work_(Minutes),VAC_RATE,OCC_RATE,PER_INCOME_0-15,PER_INCOME_15-50,PER_INCOME_50-100,PER_INCOME_100+,AVG_HH_SIZE,PER_HH_0VEH,POP_DEN_ACS,HH_DEN_ACS,HU_DEN_ACS,Average Hourly Price,parking_rate,PRICE1HR,PRICE2HR,PRICE3HR,PRICE4HR,PRICE12HR,PRICEDAY,PRICEDAYDISCOUNT,BART_FROMS,BART_TOS,CALTRAIN_OFF,CALTRAIN_ON,MUNI_RAIL_ALIGHTINGS,MUNI_RAIL_BOARDINGS,EDHLTH_RAC_SCALED,EDHLTH_WAC_SCALED,EMP_RAC_SCALED,EMP_WAC_SCALED,LEISER_RAC_SCALED,LEISER_WAC_SCALED,OTHER_RAC_SCALED,OTHER_WAC_SCALED,RETAIL_RAC_SCALED,RETAIL_WAC_SCALED,HOUSING_09_SCALED,NUM_BUS_STOPS,Count_,EDD_EMP,FREQ_S


In [196]:
estimate.columns

Index(['MONTH', 'DOW', 'AGENCY_ID', 'ROUTE_SHORT_NAME', 'DIR', 'SEQ',
       'NUMDAYS', 'OBSDAYS', 'TRIP_STOPS', 'OBS_TRIP_STOPS', 'IMP_TRIP_STOPS',
       'WGT_TRIP_STOPS', 'STOP_ID', 'ROUTE_LONG_NAME', 'ROUTE_TYPE',
       'TRIP_HEADSIGN', 'HEADWAY_S', 'FARE', 'STOPNAME', 'STOPNAME_AVL', 'EOL',
       'SOL', 'TIMEPOINT', 'ARRIVAL_TIME_DEV', 'DEPARTURE_TIME_DEV', 'DWELL_S',
       'DWELL', 'RUNTIME_S', 'RUNTIME', 'TOTTIME_S', 'TOTTIME', 'SERVMILES_S',
       'SERVMILES', 'RUNSPEED_S', 'RUNSPEED', 'TOTSPEED_S', 'TOTSPEED',
       'ONTIME5', 'ON', 'OFF', 'LOAD_ARR', 'LOAD_DEP', 'PASSMILES',
       'PASSHOURS', 'WAITHOURS', 'FULLFARE_REV', 'PASSDELAY_DEP',
       'PASSDELAY_ARR', 'RDBRDNGS', 'DOORCYCLES', 'WHEELCHAIR', 'BIKERACK',
       'CAPACITY', 'VC', 'CROWDED', 'CROWDHOURS', 'Total_Pop',
       'Total_Housing_Units',
       'Workers_16_And_Over_Mean_Travel_Time_To_Work_(Minutes)', 'VAC_RATE',
       'OCC_RATE', 'PER_INCOME_0-15', 'PER_INCOME_15-50', 'PER_INCOME_50-100',
       'PER_

In [14]:
estimate = estimate.sort(axis = 1)

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(....) is deprecated, use sort_index(.....)
  if __name__ == '__main__':


In [15]:
for column in estimate.columns:
    print(column)

AGENCY_ID
ARRIVAL_TIME_DEV
AVG_HH_SIZE
Average Hourly Price
BART_FROMS
BART_TOS
BIKERACK
CALTRAIN_OFF
CALTRAIN_ON
CAPACITY
CROWDED
CROWDHOURS
Count_
DEPARTURE_TIME_DEV
DIR
DOORCYCLES
DOW
DWELL
DWELL_S
EDD_EMP
EDHLTH_RAC_SCALED
EDHLTH_WAC_SCALED
EMP_RAC_SCALED
EMP_WAC_SCALED
EOL
FARE
FREQ_S
FULLFARE_REV
HEADWAY_S
HH_DEN_ACS
HOUSING_09_SCALED
HU_DEN_ACS
IMP_TRIP_STOPS
LEISER_RAC_SCALED
LEISER_WAC_SCALED
LOAD_ARR
LOAD_DEP
MONTH
MUNI_RAIL_ALIGHTINGS
MUNI_RAIL_BOARDINGS
NUMDAYS
NUM_BUS_STOPS
OBSDAYS
OBS_TRIP_STOPS
OCC_RATE
OFF
ON
ONTIME5
OTHER_RAC_SCALED
OTHER_WAC_SCALED
PASSDELAY_ARR
PASSDELAY_DEP
PASSHOURS
PASSMILES
PER_HH_0VEH
PER_INCOME_0-15
PER_INCOME_100+
PER_INCOME_15-50
PER_INCOME_50-100
POP_DEN_ACS
PRICE12HR
PRICE1HR
PRICE2HR
PRICE3HR
PRICE4HR
PRICEDAY
PRICEDAYDISCOUNT
RDBRDNGS
RETAIL_RAC_SCALED
RETAIL_WAC_SCALED
ROUTE_LONG_NAME
ROUTE_SHORT_NAME
ROUTE_TYPE
RUNSPEED
RUNSPEED_S
RUNTIME
RUNTIME_S
SEQ
SERVMILES
SERVMILES_S
SOL
STOPNAME
STOPNAME_AVL
STOP_ID
TIMEPOINT
TOTSPEED
TOTSPEED_S

In [16]:
change = ['Average Hourly Price',
'Count_',
'HOUSING_09_SCALED',
'LEISER_RAC_SCALED',
'LEISER_WAC_SCALED',
'OTHER_RAC_SCALED',
'OTHER_WAC_SCALED',
'PER_HH_0VEH',
'PER_INCOME_0-15',
'PER_INCOME_100+',
'PER_INCOME_15-50',
'PER_INCOME_50-100',
'PRICE12HR',
'PRICE1HR',
'PRICE2HR',
'PRICE3HR',
'PRICE4HR',
'PRICEDAY',
'PRICEDAYDISCOUNT',
'RETAIL_RAC_SCALED',
'RETAIL_WAC_SCALED',
'Workers_16_And_Over_Mean_Travel_Time_To_Work_(Minutes)',
'parking_rate']

In [17]:
changes = ['HOURLY_AVG_ON_PARK',
'TRANSBAY',
'EDHLTH_RAC_SCALED',
'EDHLTH_WAC_SCALED',
'EMP_RAC_SCALED',
'EMP_WAC_SCALED',
'HOUSING_09_SCALED',
'LEISER_RAC_SCALED',
'LEISER_WAC_SCALED',
'SHR_HH_0VEH',
'SHR_INCOME_0_15',
'SHR_INCOME_100P',
'SHR_INCOME_15_50',
'SHR_INCOME_50_100',
'12_HR_OFF_PUB_PARK',
'1_HR_OFF_PUB_PARK',
'2_HR_OFF_PUB_PARK',
'3_HR_OFF_PUB_PARK',
'4_HR_OFF_PUB_PARK',
'DAY_OFF_PUB_PARK',
'DAY_DISCOUNT_OFF_PUB_PARK',
'RETAIL_RAC_SCALED',
'RETAIL_WAC_SCALED',
'AVG_TT_TO_WORK_MINUTES',
'PARKING_RATE_OFF_RESIDENTIAL']

In [18]:
def rename_column(old_columns,new_columns,df):
    num = len(old_columns)
    count = 0
    while count < num:
        df[new_columns[count]] = df[old_columns[count]]
        df = df.drop(old_columns[count],axis = 1)
        
        count = count + 1
    return df

In [19]:
estimate = rename_column(change,changes,estimate)

In [20]:
# basic set of initial variables
mod = smf.formula.ols(formula="ON \
                       ~ EMP_WAC_SCALED \
                       + HOUSING_09_SCALED\
                       + ONTIME5 \
                       + HEADWAY_S \
                       + EOL\
                       + SOL \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                     ON   R-squared:                       0.323
Model:                            OLS   Adj. R-squared:                  0.323
Method:                 Least Squares   F-statistic:                     1347.
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:47:10   Log-Likelihood:            -1.0925e+05
No. Observations:               16940   AIC:                         2.185e+05
Df Residuals:                   16934   BIC:                         2.186e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
EMP_WAC_SCALED        0.1151      0.00

In [21]:
# basic set of initial variables
mod = smf.formula.ols(formula="ON \
                       ~ EMP_WAC_SCALED \
                       + HOUSING_09_SCALED\
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                     ON   R-squared:                       0.302
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                     1220.
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:47:20   Log-Likelihood:            -1.0951e+05
No. Observations:               16940   AIC:                         2.190e+05
Df Residuals:                   16934   BIC:                         2.191e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
EMP_WAC_SCALED        0.0686      0.00

In [22]:
# basic set of initial variables
mod = smf.formula.ols(formula="ON \
                       ~ EMP_WAC_SCALED \
                       + HOUSING_09_SCALED\
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                     ON   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.358
Method:                 Least Squares   F-statistic:                     1348.
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:47:29   Log-Likelihood:            -1.0880e+05
No. Observations:               16940   AIC:                         2.176e+05
Df Residuals:                   16933   BIC:                         2.177e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
EMP_WAC_SCALED        0.0392      0.00

In [23]:
# add competing stops
mod = smf.formula.ols(formula="ON \
                       ~ EMP_WAC_SCALED \
                       + HOUSING_09_SCALED\
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + NUM_BUS_STOPS \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                     ON   R-squared:                       0.360
Model:                            OLS   Adj. R-squared:                  0.359
Method:                 Least Squares   F-statistic:                     1189.
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:47:33   Log-Likelihood:            -1.0877e+05
No. Observations:               16940   AIC:                         2.176e+05
Df Residuals:                   16932   BIC:                         2.176e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
EMP_WAC_SCALED        0.0265      0.00

In [24]:
# add transbay terminal 
mod = smf.formula.ols(formula="ON \
                       ~ EMP_WAC_SCALED \
                       + HOUSING_09_SCALED\
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + NUM_BUS_STOPS \
                       + TRANSBAY\
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                     ON   R-squared:                       0.361
Model:                            OLS   Adj. R-squared:                  0.360
Method:                 Least Squares   F-statistic:                     1061.
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:47:37   Log-Likelihood:            -1.0876e+05
No. Observations:               16940   AIC:                         2.175e+05
Df Residuals:                   16931   BIC:                         2.176e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
EMP_WAC_SCALED        0.0278      0.00

In [25]:
#add in BARTS Boardings 
mod = smf.formula.ols(formula="ON \
                       ~ EMP_WAC_SCALED \
                       + HOUSING_09_SCALED\
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + NUM_BUS_STOPS \
                       + BART_FROMS \
                       + TRANSBAY \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                     ON   R-squared:                       0.372
Model:                            OLS   Adj. R-squared:                  0.371
Method:                 Least Squares   F-statistic:                     1001.
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:47:39   Log-Likelihood:            -1.0862e+05
No. Observations:               16940   AIC:                         2.173e+05
Df Residuals:                   16930   BIC:                         2.173e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
EMP_WAC_SCALED        0.0394      0.00

In [26]:
# swap out hh density
mod = smf.formula.ols(formula="ON \
                       ~ EMP_WAC_SCALED \
                       + HH_DEN_ACS \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + NUM_BUS_STOPS \
                       + BART_FROMS \
                       + TRANSBAY \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                     ON   R-squared:                       0.373
Model:                            OLS   Adj. R-squared:                  0.373
Method:                 Least Squares   F-statistic:                     1007.
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:47:39   Log-Likelihood:            -1.0860e+05
No. Observations:               16940   AIC:                         2.172e+05
Df Residuals:                   16930   BIC:                         2.173e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
EMP_WAC_SCALED    -0.0099      0.011     -0.

In [27]:
# Drop bart boardings
mod = smf.formula.ols(formula="ON \
                       ~ EMP_WAC_SCALED \
                       + HH_DEN_ACS \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + NUM_BUS_STOPS \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                     ON   R-squared:                       0.362
Model:                            OLS   Adj. R-squared:                  0.362
Method:                 Least Squares   F-statistic:                     1200.
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:47:39   Log-Likelihood:            -1.0875e+05
No. Observations:               16940   AIC:                         2.175e+05
Df Residuals:                   16932   BIC:                         2.176e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
EMP_WAC_SCALED    -0.0324      0.011     -3.

In [28]:
# drop competing stops
mod = smf.formula.ols(formula="ON \
                       ~ EMP_WAC_SCALED \
                       + HH_DEN_ACS \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                     ON   R-squared:                       0.308
Model:                            OLS   Adj. R-squared:                  0.308
Method:                 Least Squares   F-statistic:                     1255.
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:47:40   Log-Likelihood:            -1.0944e+05
No. Observations:               16940   AIC:                         2.189e+05
Df Residuals:                   16934   BIC:                         2.189e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
EMP_WAC_SCALED    -0.0734      0.011     -6.

In [29]:
# RUN 20 
# back to preferred model
mod = smf.formula.ols(formula="ON \
                       ~ EMP_WAC_SCALED \
                       + HH_DEN_ACS \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + NUM_BUS_STOPS \
                       + TRANSBAY \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                     ON   R-squared:                       0.363
Model:                            OLS   Adj. R-squared:                  0.362
Method:                 Least Squares   F-statistic:                     1071.
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:47:40   Log-Likelihood:            -1.0874e+05
No. Observations:               16940   AIC:                         2.175e+05
Df Residuals:                   16931   BIC:                         2.176e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
EMP_WAC_SCALED    -0.0301      0.011     -2.

In [30]:
estimate[np.isnan(estimate['AVG_HH_SIZE'])]

,AGENCY_ID,ARRIVAL_TIME_DEV,AVG_HH_SIZE,BART_FROMS,BART_TOS,BIKERACK,CALTRAIN_OFF,CALTRAIN_ON,CAPACITY,CROWDED,CROWDHOURS,DEPARTURE_TIME_DEV,DIR,DOORCYCLES,DOW,DWELL,DWELL_S,EDD_EMP,EDHLTH_RAC_SCALED,EDHLTH_WAC_SCALED,EMP_RAC_SCALED,EMP_WAC_SCALED,EOL,FARE,FREQ_S,FULLFARE_REV,HEADWAY_S,HH_DEN_ACS,HU_DEN_ACS,IMP_TRIP_STOPS,LOAD_ARR,LOAD_DEP,MONTH,MUNI_RAIL_ALIGHTINGS,MUNI_RAIL_BOARDINGS,NUMDAYS,NUM_BUS_STOPS,OBSDAYS,OBS_TRIP_STOPS,OCC_RATE,OFF,ON,ONTIME5,PASSDELAY_ARR,PASSDELAY_DEP,PASSHOURS,PASSMILES,POP_DEN_ACS,RDBRDNGS,ROUTE_LONG_NAME,ROUTE_SHORT_NAME,ROUTE_TYPE,RUNSPEED,RUNSPEED_S,RUNTIME,RUNTIME_S,SEQ,SERVMILES,SERVMILES_S,SOL,STOPNAME,STOPNAME_AVL,STOP_ID,TIMEPOINT,TOTSPEED,TOTSPEED_S,TOTTIME,TOTTIME_S,TRIP_HEADSIGN,TRIP_STOPS,Total_Housing_Units,Total_Pop,VAC_RATE,VC,WAITHOURS,WGT_TRIP_STOPS,WHEELCHAIR,HOURLY_AVG_ON_PARK,TRANSBAY,HOUSING_09_SCALED,LEISER_RAC_SCALED,LEISER_WAC_SCALED,SHR_HH_0VEH,SHR_INCOME_0_15,SHR_INCOME_100P,SHR_INCOME_15_50,SHR_INCOME_50_100,12_HR_OFF_PUB_PARK,1_HR_OFF_PUB_PARK,2_HR_OFF_PUB_PARK,3_HR_OFF_PUB_PARK,4_HR_OFF_PUB_PARK,DAY_OFF_PUB_PARK,DAY_DISCOUNT_OFF_PUB_PARK,RETAIL_RAC_SCALED,RETAIL_WAC_SCALED


In [31]:
for column in estimate.columns:
    print(column)

AGENCY_ID
ARRIVAL_TIME_DEV
AVG_HH_SIZE
BART_FROMS
BART_TOS
BIKERACK
CALTRAIN_OFF
CALTRAIN_ON
CAPACITY
CROWDED
CROWDHOURS
DEPARTURE_TIME_DEV
DIR
DOORCYCLES
DOW
DWELL
DWELL_S
EDD_EMP
EDHLTH_RAC_SCALED
EDHLTH_WAC_SCALED
EMP_RAC_SCALED
EMP_WAC_SCALED
EOL
FARE
FREQ_S
FULLFARE_REV
HEADWAY_S
HH_DEN_ACS
HU_DEN_ACS
IMP_TRIP_STOPS
LOAD_ARR
LOAD_DEP
MONTH
MUNI_RAIL_ALIGHTINGS
MUNI_RAIL_BOARDINGS
NUMDAYS
NUM_BUS_STOPS
OBSDAYS
OBS_TRIP_STOPS
OCC_RATE
OFF
ON
ONTIME5
PASSDELAY_ARR
PASSDELAY_DEP
PASSHOURS
PASSMILES
POP_DEN_ACS
RDBRDNGS
ROUTE_LONG_NAME
ROUTE_SHORT_NAME
ROUTE_TYPE
RUNSPEED
RUNSPEED_S
RUNTIME
RUNTIME_S
SEQ
SERVMILES
SERVMILES_S
SOL
STOPNAME
STOPNAME_AVL
STOP_ID
TIMEPOINT
TOTSPEED
TOTSPEED_S
TOTTIME
TOTTIME_S
TRIP_HEADSIGN
TRIP_STOPS
Total_Housing_Units
Total_Pop
VAC_RATE
VC
WAITHOURS
WGT_TRIP_STOPS
WHEELCHAIR
HOURLY_AVG_ON_PARK
TRANSBAY
HOUSING_09_SCALED
LEISER_RAC_SCALED
LEISER_WAC_SCALED
SHR_HH_0VEH
SHR_INCOME_0_15
SHR_INCOME_100P
SHR_INCOME_15_50
SHR_INCOME_50_100
12_HR_OFF_PUB_PARK
1

In [214]:
estimate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17225 entries, 0 to 18091
Data columns (total 96 columns):
AGENCY_ID                    17225 non-null object
ARRIVAL_TIME_DEV             17225 non-null float64
AVG_HH_SIZE                  17225 non-null float64
BART_FROMS                   17225 non-null float64
BART_TOS                     17225 non-null float64
BIKERACK                     17225 non-null float64
CALTRAIN_OFF                 17225 non-null float64
CALTRAIN_ON                  17225 non-null float64
CAPACITY                     17225 non-null float64
CROWDED                      17225 non-null float64
CROWDHOURS                   17225 non-null float64
DEPARTURE_TIME_DEV           17225 non-null float64
DIR                          17225 non-null int64
DOORCYCLES                   17225 non-null float64
DOW                          17225 non-null int64
DWELL                        17225 non-null float64
DWELL_S                      17225 non-null float64
EDD_EMP     

In [215]:
check = estimate[np.isnan(estimate['EDD_EMP'])]

In [ ]:
check.to_csv('check.csv')

In [ ]:
# look at pair plots
pairing_estimate = pd.DataFrame()
convert = ['ON','EMP_WAC_SCALED','AVG_HH_SIZE','HOURLY_AVG_ON_PARK','BART_FROMS','BART_TOS','EDD_EMP','FARE','FREQ_S','HU_DEN_ACS','HOUSING_09_SCALED','HH_DEN_ACS','ONTIME5','SHR_HH_0VEH','SHR_INCOME_100P','SHR_INCOME_0_15','RUNSPEED','SERVMILES_S','VAC_RATE','MUNI_RAIL_BOARDINGS']
pairing_estimate = estimate[convert].dropna()
print(len(pairing_estimate))
fig = sns.pairplot(pairing_estimate[['ON','EMP_WAC_SCALED','AVG_HH_SIZE','BART_FROMS','BART_TOS','EDD_EMP','FREQ_S','HU_DEN_ACS','HOUSING_09_SCALED','HH_DEN_ACS','ONTIME5','SHR_HH_0VEH','SHR_INCOME_100P','SHR_INCOME_0_15','RUNSPEED','SERVMILES_S','VAC_RATE','MUNI_RAIL_BOARDINGS']])
fig.savefig('pair_plot.jpg')

16737


In [157]:
for column in estimate.columns:
    print(column)

AGENCY_ID
ARRIVAL_TIME_DEV
AVG_HH_SIZE
BART_FROMS
BART_TOS
BIKERACK
CALTRAIN_OFF
CALTRAIN_ON
CAPACITY
CROWDED
CROWDHOURS
DEPARTURE_TIME_DEV
DIR
DOORCYCLES
DOW
DWELL
DWELL_S
EDD_EMP
EDHLTH_RAC_SCALED
EDHLTH_WAC_SCALED
EMP_RAC_SCALED
EMP_WAC_SCALED
EOL
FARE
FREQ_S
FULLFARE_REV
HEADWAY_S
HH_DEN_ACS
HU_DEN_ACS
IMP_TRIP_STOPS
LOAD_ARR
LOAD_DEP
MONTH
MUNI_RAIL_ALIGHTINGS
MUNI_RAIL_BOARDINGS
NUMDAYS
NUM_BUS_STOPS
OBSDAYS
OBS_TRIP_STOPS
OCC_RATE
OFF
ON
ONTIME5
PASSDELAY_ARR
PASSDELAY_DEP
PASSHOURS
PASSMILES
POP_DEN_ACS
RDBRDNGS
ROUTE_LONG_NAME
ROUTE_SHORT_NAME
ROUTE_TYPE
RUNSPEED
RUNSPEED_S
RUNTIME
RUNTIME_S
SEQ
SERVMILES
SERVMILES_S
SOL
STOPNAME
STOPNAME_AVL
STOP_ID
TIMEPOINT
TOTSPEED
TOTSPEED_S
TOTTIME
TOTTIME_S
TRIP_HEADSIGN
TRIP_STOPS
Total_Housing_Units
Total_Pop
VAC_RATE
VC
WAITHOURS
WGT_TRIP_STOPS
WHEELCHAIR
HOURLY_AVG_ON_PARK
TRANSBAY
HOUSING_09_SCALED
LEISER_RAC_SCALED
LEISER_WAC_SCALED
SHR_HH_0VEH
SHR_INCOME_0_15
SHR_INCOME_100P
SHR_INCOME_15_50
SHR_INCOME_50_100
12_HR_OFF_PUB_PARK
1

In [32]:
# RUN 21
# back to preferred model, but with missing data dropped
mod = smf.formula.ols(formula="ON \
                       ~ EMP_WAC_SCALED \
                       + HH_DEN_ACS \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + NUM_BUS_STOPS \
                       + TRANSBAY \
                       + BART_FROMS \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                     ON   R-squared:                       0.373
Model:                            OLS   Adj. R-squared:                  0.373
Method:                 Least Squares   F-statistic:                     1007.
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:48:24   Log-Likelihood:            -1.0860e+05
No. Observations:               16940   AIC:                         2.172e+05
Df Residuals:                   16930   BIC:                         2.173e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
EMP_WAC_SCALED    -0.0099      0.011     -0.

In [33]:
# RUN 22
# test 0 vehicle households
mod = smf.formula.ols(formula="ON \
                       ~ EMP_WAC_SCALED \
                       + HH_DEN_ACS \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + NUM_BUS_STOPS \
                       + TRANSBAY \
                       + BART_FROMS \
                       + SHR_HH_0VEH \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                     ON   R-squared:                       0.383
Model:                            OLS   Adj. R-squared:                  0.382
Method:                 Least Squares   F-statistic:                     953.8
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:48:27   Log-Likelihood:            -1.0847e+05
No. Observations:               16940   AIC:                         2.170e+05
Df Residuals:                   16929   BIC:                         2.170e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
EMP_WAC_SCALED    -0.0023      0.011     -0.

In [34]:
# RUN 23
# test average household size
mod = smf.formula.ols(formula="ON \
                       ~ EMP_WAC_SCALED \
                       + HH_DEN_ACS \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + NUM_BUS_STOPS \
                       + TRANSBAY \
                       + BART_FROMS \
                       + SHR_HH_0VEH \
                       + AVG_HH_SIZE \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                     ON   R-squared:                       0.383
Model:                            OLS   Adj. R-squared:                  0.382
Method:                 Least Squares   F-statistic:                     875.0
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:48:28   Log-Likelihood:            -1.0846e+05
No. Observations:               16940   AIC:                         2.170e+05
Df Residuals:                   16928   BIC:                         2.170e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
EMP_WAC_SCALED    -0.0084      0.011     -0.

In [35]:
# RUN 23
# test pop den instead of hh den
mod = smf.formula.ols(formula="ON \
                       ~ EMP_WAC_SCALED \
                       + POP_DEN_ACS \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + NUM_BUS_STOPS \
                       + TRANSBAY \
                       + BART_FROMS \
                       + SHR_HH_0VEH \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                     ON   R-squared:                       0.384
Model:                            OLS   Adj. R-squared:                  0.383
Method:                 Least Squares   F-statistic:                     957.7
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:48:28   Log-Likelihood:            -1.0845e+05
No. Observations:               16940   AIC:                         2.169e+05
Df Residuals:                   16929   BIC:                         2.170e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
EMP_WAC_SCALED    -0.0255      0.011     -2.

In [36]:
# RUN 25
# test housing units for consistency with change data
mod = smf.formula.ols(formula="ON \
                       ~ EMP_WAC_SCALED \
                       + HU_DEN_ACS \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + NUM_BUS_STOPS \
                       + TRANSBAY \
                       + BART_FROMS \
                       + SHR_HH_0VEH \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                     ON   R-squared:                       0.382
Model:                            OLS   Adj. R-squared:                  0.382
Method:                 Least Squares   F-statistic:                     953.0
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:48:28   Log-Likelihood:            -1.0847e+05
No. Observations:               16940   AIC:                         2.170e+05
Df Residuals:                   16929   BIC:                         2.170e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
EMP_WAC_SCALED     0.0036      0.010      0.

In [37]:
# RUN 26
# test housing units for consistency with change data -- at census block level
mod = smf.formula.ols(formula="ON \
                       ~ EMP_WAC_SCALED \
                       + HOUSING_09_SCALED \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + NUM_BUS_STOPS \
                       + TRANSBAY \
                       + BART_FROMS \
                       + SHR_HH_0VEH \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                     ON   R-squared:                       0.383
Model:                            OLS   Adj. R-squared:                  0.383
Method:                 Least Squares   F-statistic:                     956.0
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:48:28   Log-Likelihood:            -1.0846e+05
No. Observations:               16940   AIC:                         2.169e+05
Df Residuals:                   16929   BIC:                         2.170e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
EMP_WAC_SCALED        0.0185      0.00

In [38]:
for column in estimate.columns.sort_values():
    print(column)

12_HR_OFF_PUB_PARK
1_HR_OFF_PUB_PARK
2_HR_OFF_PUB_PARK
3_HR_OFF_PUB_PARK
4_HR_OFF_PUB_PARK
AGENCY_ID
ARRIVAL_TIME_DEV
AVG_HH_SIZE
BART_FROMS
BART_TOS
BIKERACK
CALTRAIN_OFF
CALTRAIN_ON
CAPACITY
CROWDED
CROWDHOURS
DAY_DISCOUNT_OFF_PUB_PARK
DAY_OFF_PUB_PARK
DEPARTURE_TIME_DEV
DIR
DOORCYCLES
DOW
DWELL
DWELL_S
EDD_EMP
EDHLTH_RAC_SCALED
EDHLTH_WAC_SCALED
EMP_RAC_SCALED
EMP_WAC_SCALED
EOL
FARE
FREQ_S
FULLFARE_REV
HEADWAY_S
HH_DEN_ACS
HOURLY_AVG_ON_PARK
HOUSING_09_SCALED
HU_DEN_ACS
IMP_TRIP_STOPS
LEISER_RAC_SCALED
LEISER_WAC_SCALED
LOAD_ARR
LOAD_DEP
MONTH
MUNI_RAIL_ALIGHTINGS
MUNI_RAIL_BOARDINGS
NUMDAYS
NUM_BUS_STOPS
OBSDAYS
OBS_TRIP_STOPS
OCC_RATE
OFF
ON
ONTIME5
PASSDELAY_ARR
PASSDELAY_DEP
PASSHOURS
PASSMILES
POP_DEN_ACS
RDBRDNGS
RETAIL_RAC_SCALED
RETAIL_WAC_SCALED
ROUTE_LONG_NAME
ROUTE_SHORT_NAME
ROUTE_TYPE
RUNSPEED
RUNSPEED_S
RUNTIME
RUNTIME_S
SEQ
SERVMILES
SERVMILES_S
SHR_HH_0VEH
SHR_INCOME_0_15
SHR_INCOME_100P
SHR_INCOME_15_50
SHR_INCOME_50_100
SOL
STOPNAME
STOPNAME_AVL
STOP_ID
TIMEPOINT


In [39]:
# RUN 27
# add income groups
mod = smf.formula.ols(formula="ON \
                       ~ EMP_WAC_SCALED \
                       + HOUSING_09_SCALED \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + NUM_BUS_STOPS \
                       + TRANSBAY \
                       + BART_FROMS \
                       + SHR_HH_0VEH \
                       + SHR_INCOME_0_15 \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                     ON   R-squared:                       0.383
Model:                            OLS   Adj. R-squared:                  0.383
Method:                 Least Squares   F-statistic:                     876.3
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:48:29   Log-Likelihood:            -1.0846e+05
No. Observations:               16940   AIC:                         2.169e+05
Df Residuals:                   16928   BIC:                         2.170e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
EMP_WAC_SCALED        0.0188      0.00

In [40]:
# RUN 27
# add income groups
mod = smf.formula.ols(formula="ON \
                       ~ EMP_WAC_SCALED \
                       + HOUSING_09_SCALED \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + NUM_BUS_STOPS \
                       + TRANSBAY \
                       + BART_FROMS \
                       + SHR_HH_0VEH \
                       + SHR_INCOME_0_15 \
                       + SHR_INCOME_15_50 \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                     ON   R-squared:                       0.375
Model:                            OLS   Adj. R-squared:                  0.372
Method:                 Least Squares   F-statistic:                     126.7
Date:                Fri, 14 Jul 2017   Prob (F-statistic):          2.09e-268
Time:                        11:48:29   Log-Likelihood:                -18422.
No. Observations:                2754   AIC:                         3.687e+04
Df Residuals:                    2741   BIC:                         3.695e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
EMP_WAC_SCALED       -0.0805      0.02

In [41]:
# RUN 27
# add income groups and drop competing stops because of a high p value (low t stat)
mod = smf.formula.ols(formula="ON \
                       ~ EMP_WAC_SCALED \
                       + HOUSING_09_SCALED \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + TRANSBAY \
                       + BART_FROMS \
                       + SHR_HH_0VEH \
                       + SHR_INCOME_100P \
                       -1",
             data =estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                     ON   R-squared:                       0.381
Model:                            OLS   Adj. R-squared:                  0.381
Method:                 Least Squares   F-statistic:                     949.1
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:48:29   Log-Likelihood:            -1.0848e+05
No. Observations:               16940   AIC:                         2.170e+05
Df Residuals:                   16929   BIC:                         2.171e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
EMP_WAC_SCALED        0.0339      0.00

In [42]:
# RUN 30
# add income groups
mod = smf.formula.ols(formula="ON \
                       ~ EMP_WAC_SCALED \
                       + HOUSING_09_SCALED \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + TRANSBAY \
                       + BART_FROMS \
                       + SHR_HH_0VEH \
                       + SHR_INCOME_0_15 \
                       + SHR_INCOME_100P \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                     ON   R-squared:                       0.382
Model:                            OLS   Adj. R-squared:                  0.381
Method:                 Least Squares   F-statistic:                     870.2
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:48:29   Log-Likelihood:            -1.0848e+05
No. Observations:               16940   AIC:                         2.170e+05
Df Residuals:                   16928   BIC:                         2.171e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
EMP_WAC_SCALED        0.0317      0.00

In [43]:
for column in estimate.columns.sort_values():
    print(column)

12_HR_OFF_PUB_PARK
1_HR_OFF_PUB_PARK
2_HR_OFF_PUB_PARK
3_HR_OFF_PUB_PARK
4_HR_OFF_PUB_PARK
AGENCY_ID
ARRIVAL_TIME_DEV
AVG_HH_SIZE
BART_FROMS
BART_TOS
BIKERACK
CALTRAIN_OFF
CALTRAIN_ON
CAPACITY
CROWDED
CROWDHOURS
DAY_DISCOUNT_OFF_PUB_PARK
DAY_OFF_PUB_PARK
DEPARTURE_TIME_DEV
DIR
DOORCYCLES
DOW
DWELL
DWELL_S
EDD_EMP
EDHLTH_RAC_SCALED
EDHLTH_WAC_SCALED
EMP_RAC_SCALED
EMP_WAC_SCALED
EOL
FARE
FREQ_S
FULLFARE_REV
HEADWAY_S
HH_DEN_ACS
HOURLY_AVG_ON_PARK
HOUSING_09_SCALED
HU_DEN_ACS
IMP_TRIP_STOPS
LEISER_RAC_SCALED
LEISER_WAC_SCALED
LOAD_ARR
LOAD_DEP
MONTH
MUNI_RAIL_ALIGHTINGS
MUNI_RAIL_BOARDINGS
NUMDAYS
NUM_BUS_STOPS
OBSDAYS
OBS_TRIP_STOPS
OCC_RATE
OFF
ON
ONTIME5
PASSDELAY_ARR
PASSDELAY_DEP
PASSHOURS
PASSMILES
POP_DEN_ACS
RDBRDNGS
RETAIL_RAC_SCALED
RETAIL_WAC_SCALED
ROUTE_LONG_NAME
ROUTE_SHORT_NAME
ROUTE_TYPE
RUNSPEED
RUNSPEED_S
RUNTIME
RUNTIME_S
SEQ
SERVMILES
SERVMILES_S
SHR_HH_0VEH
SHR_INCOME_0_15
SHR_INCOME_100P
SHR_INCOME_15_50
SHR_INCOME_50_100
SOL
STOPNAME
STOPNAME_AVL
STOP_ID
TIMEPOINT


In [45]:
# RUN
# these are the basic variables we want, so let's start from here...
mod = smf.formula.ols(formula="ON \
                       ~ EMP_WAC_SCALED \
                       + HOUSING_09_SCALED \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + NUM_BUS_STOPS \
                       + TRANSBAY \
                       + BART_FROMS \
                       + SHR_HH_0VEH \
                       + SHR_INCOME_100P \
                       + MUNI_RAIL_BOARDINGS \
                       + CALTRAIN_ON \
                       + PARKING_RATE_OFF_RESIDENTIAL \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

PatsyError: Error evaluating factor: NameError: name 'PARKING_RATE_OFF_RESIDENTIAL' is not defined
    ON                        ~ EMP_WAC_SCALED                        + HOUSING_09_SCALED                        + ONTIME5                        + FREQ_S                        + EOL                       + SOL                        + RUNSPEED                        + NUM_BUS_STOPS                        + TRANSBAY                        + BART_FROMS                        + SHR_HH_0VEH                        + SHR_INCOME_100P                        + MUNI_RAIL_BOARDINGS                        + CALTRAIN_ON                        + PARKING_RATE_OFF_RESIDENTIAL                        -1
                                                                                                                                                                                                                                                                                                                  

In [46]:
# try average of ONS and OFFS
estimate['AVG_RIDE'] = (estimate['ON'] + estimate['OFF'])/2

In [47]:
estimate.head()

,AGENCY_ID,ARRIVAL_TIME_DEV,AVG_HH_SIZE,BART_FROMS,BART_TOS,BIKERACK,CALTRAIN_OFF,CALTRAIN_ON,CAPACITY,CROWDED,CROWDHOURS,DEPARTURE_TIME_DEV,DIR,DOORCYCLES,DOW,DWELL,DWELL_S,EDD_EMP,EDHLTH_RAC_SCALED,EDHLTH_WAC_SCALED,EMP_RAC_SCALED,EMP_WAC_SCALED,EOL,FARE,FREQ_S,FULLFARE_REV,HEADWAY_S,HH_DEN_ACS,HU_DEN_ACS,IMP_TRIP_STOPS,LOAD_ARR,LOAD_DEP,MONTH,MUNI_RAIL_ALIGHTINGS,MUNI_RAIL_BOARDINGS,NUMDAYS,NUM_BUS_STOPS,OBSDAYS,OBS_TRIP_STOPS,OCC_RATE,OFF,ON,ONTIME5,PASSDELAY_ARR,PASSDELAY_DEP,PASSHOURS,PASSMILES,POP_DEN_ACS,RDBRDNGS,ROUTE_LONG_NAME,ROUTE_SHORT_NAME,ROUTE_TYPE,RUNSPEED,RUNSPEED_S,RUNTIME,RUNTIME_S,SEQ,SERVMILES,SERVMILES_S,SOL,STOPNAME,STOPNAME_AVL,STOP_ID,TIMEPOINT,TOTSPEED,TOTSPEED_S,TOTTIME,TOTTIME_S,TRIP_HEADSIGN,TRIP_STOPS,Total_Housing_Units,Total_Pop,VAC_RATE,VC,WAITHOURS,WGT_TRIP_STOPS,WHEELCHAIR,HOURLY_AVG_ON_PARK,TRANSBAY,HOUSING_09_SCALED,LEISER_RAC_SCALED,LEISER_WAC_SCALED,SHR_HH_0VEH,SHR_INCOME_0_15,SHR_INCOME_100P,SHR_INCOME_15_50,SHR_INCOME_50_100,12_HR_OFF_PUB_PARK,1_HR_OFF_PUB_PARK,2_HR_OFF_PUB_PARK,3_HR_OFF_PUB_PARK,4_HR_OFF_PUB_PARK,DAY_OFF_PUB_PARK,DAY_DISCOUNT_OFF_PUB_PARK,RETAIL_RAC_SCALED,RETAIL_WAC_SCALED,AVG_RIDE
0,SFMTA,-3.875743,1.584002,4673.0,4772.0,0.0,0.0,0.0,189.0,0.0,0.0,-0.070542,0,21.049339,1,49.476280,0.0,7134.0,717.267880,32.287304,963.525560,144.291088,0,2.0,5.625,516.584921,10.666667,81.440025,97.511565,0.0,0.240741,258.292460,2009-10-01,2496.0,4136.0,22,10.0,18.0,2,0.835183,0.240741,258.292460,0.737434,91.157255,287.502463,20.102898,0.000000,129.001163,0.0,NORIEGA A EXP,016AX,3,0.00,0.00,0.000000,0.00,3,0.000,0.000,1,CYRIL MAGNIN ST & MARKET ST,5TH ST NORTH&MARKET ST NW,3612,1.0,0.00,0.00,49.476280,0.00,48TH AVENUE,13,4775.0,6317.0,0.164817,0.000294,23.016448,13.0,0.000000,350.0,0.0,1437.649130,0.912738,0.650953,0.003761,0.278084,0.067202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.313950,1468.019464,29.4,0.320246,129.266601
2,SFMTA,0.270194,1.584002,0.0,0.0,0.0,0.0,0.0,189.0,0.0,0.0,0.350709,0,6.580820,1,1.049097,0.0,1061.0,1625.214651,90.333869,220.841684,315.532142,0,2.0,5.625,14.258730,10.666667,81.440025,97.511565,0.0,258.292460,265.181085,2009-10-01,0.0,0.0,22,8.0,18.0,2,0.835183,0.240741,7.129365,0.728175,5.569709,5.972606,6.700418,25.570954,129.001163,0.0,NORIEGA A EXP,016AX,3,3.95,6.99,18.514923,11.05,5,1.287,1.287,0,TURK ST & TAYLOR ST,TURK ST&TAYLOR ST N-MB/BZ,6733,0.0,3.95,6.99,19.564020,11.05,48TH AVENUE,13,4775.0,6317.0,0.164817,0.315375,0.531593,13.0,0.361111,350.0,0.0,1123.327688,0.912738,0.650953,0.003761,0.278084,0.067202,1150.0,1150.0,1150.0,1150.0,1150.0,1150.0,995.5,43.447719,113.747395,29.4,0.167471,3.685053
3,SFMTA,0.134795,1.584002,0.0,0.0,0.0,0.0,0.0,189.0,0.0,0.0,0.177869,0,3.930952,1,0.563574,0.0,838.0,2444.005159,279.076291,18.451606,857.045553,0,2.0,5.625,8.343386,10.666667,81.440025,97.511565,0.0,265.181085,268.269444,2009-10-01,0.0,0.0,22,4.0,18.0,2,0.835183,1.083333,4.171693,0.676323,6.390463,7.237389,3.611110,39.511982,129.001163,0.0,NORIEGA A EXP,016AX,3,11.03,9.12,9.969848,12.74,6,1.937,1.937,0,TURK ST & JONES ST,TURK ST&JONES ST NW-FS/BZ,6719,0.0,11.03,9.12,10.533422,12.74,48TH AVENUE,13,4775.0,6317.0,0.164817,0.323786,0.348186,13.0,0.000000,300.0,0.0,700.729429,0.912738,0.650953,0.003761,0.278084,0.067202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108.358923,18.543171,29.4,0.375314,2.627513
4,SFMTA,0.291456,1.742512,0.0,0.0,0.0,0.0,0.0,189.0,0.0,0.0,0.363253,0,5.488889,1,0.933352,0.0,1190.0,2625.683957,336.875667,18.105043,1023.465203,0,2.0,5.625,5.729630,10.666667,35.651264,40.999403,0.0,268.269444,268.100926,2009-10-01,0.0,0.0,22,7.0,18.0,2,0.869556,3.033333,2.864815,0.645238,5.081315,5.410167,3.485574,24.412519,62.122754,0.0,NORIEGA A EXP,016AX,3,6.96,9.10,9.271992,7.80,7,1.183,1.183,0,TURK ST & LEAVENWORTH ST,TURK ST&LEAVENWORTH ST NW,6721,0.0,6.96,9.10,10.205344,7.80,48TH AVENUE,13,4569.0,6923.0,0.130444,0.327557,0.345664,13.0,0.000000,300.0,0.0,557.984469,0.872892,0.420841,0.034734,0.397433,0.146992,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,1671.0,129.498961,24.314163,

In [48]:
# RUN
# test average riders

mod = smf.formula.ols(formula="AVG_RIDE \
                       ~ EMP_WAC_SCALED \
                       + HOUSING_09_SCALED \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + NUM_BUS_STOPS \
                       + TRANSBAY \
                       + BART_FROMS \
                       + SHR_HH_0VEH \
                       + SHR_INCOME_100P \
                       + MUNI_RAIL_BOARDINGS \
                       + CALTRAIN_ON \
                       + PARKING_RATE_OFF_RESIDENTIAL \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

PatsyError: Error evaluating factor: NameError: name 'PARKING_RATE_OFF_RESIDENTIAL' is not defined
    AVG_RIDE                        ~ EMP_WAC_SCALED                        + HOUSING_09_SCALED                        + ONTIME5                        + FREQ_S                        + EOL                       + SOL                        + RUNSPEED                        + NUM_BUS_STOPS                        + TRANSBAY                        + BART_FROMS                        + SHR_HH_0VEH                        + SHR_INCOME_100P                        + MUNI_RAIL_BOARDINGS                        + CALTRAIN_ON                        + PARKING_RATE_OFF_RESIDENTIAL                        -1
                                                                                                                                                                                                                                                                                                            

In [49]:
# test log of average riders
estimate['LOG_RIDERS'] = np.log(estimate.AVG_RIDE + 1)

In [50]:
# RUN
# test log average riders

mod = smf.formula.ols(formula="LOG_RIDERS \
                       ~ EMP_WAC_SCALED \
                       + HOUSING_09_SCALED \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + NUM_BUS_STOPS \
                       + TRANSBAY \
                       + BART_FROMS \
                       + SHR_HH_0VEH \
                       + SHR_INCOME_100P \
                       + MUNI_RAIL_BOARDINGS \
                       + CALTRAIN_ON \
                       + PARKING_RATE_OFF_RESIDENTIAL \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

PatsyError: Error evaluating factor: NameError: name 'PARKING_RATE_OFF_RESIDENTIAL' is not defined
    LOG_RIDERS                        ~ EMP_WAC_SCALED                        + HOUSING_09_SCALED                        + ONTIME5                        + FREQ_S                        + EOL                       + SOL                        + RUNSPEED                        + NUM_BUS_STOPS                        + TRANSBAY                        + BART_FROMS                        + SHR_HH_0VEH                        + SHR_INCOME_100P                        + MUNI_RAIL_BOARDINGS                        + CALTRAIN_ON                        + PARKING_RATE_OFF_RESIDENTIAL                        -1
                                                                                                                                                                                                                                                                                                          

In [ ]:
# RUN
# test log average riders

mod = smf.formula.ols(formula="LOG_RIDERS \
                       ~ EMP_WAC_SCALED \
                       + HOUSING_09_SCALED \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + NUM_BUS_STOPS \
                       + TRANSBAY \
                       + BART_FROMS \
                       + SHR_HH_0VEH \
                       + SHR_INCOME_100P \
                       + CALTRAIN_ON \
                       + PARKING_RATE_OFF_RESIDENTIAL \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

In [ ]:
# RUN
# test log average riders

mod = smf.formula.ols(formula="LOG_RIDERS \
                       ~ EMP_WAC_SCALED \
                       + HOUSING_09_SCALED \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + NUM_BUS_STOPS \
                       + TRANSBAY \
                       + BART_FROMS \
                       + SHR_HH_0VEH \
                       + SHR_INCOME_100P \
                       + PARKING_RATE_OFF_RESIDENTIAL \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

In [ ]:
# RUN
# try low income but the p value is worst

mod = smf.formula.ols(formula="LOG_RIDERS \
                       ~ EMP_WAC_SCALED \
                       + HOUSING_09_SCALED \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + NUM_BUS_STOPS \
                       + TRANSBAY \
                       + BART_FROMS \
                       + SHR_HH_0VEH \
                       + SHR_INCOME_0_15 \
                       + PARKING_RATE_OFF_RESIDENTIAL \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

In [51]:
# RUN
# go back to before but without parking

mod = smf.formula.ols(formula="LOG_RIDERS \
                       ~ EMP_WAC_SCALED \
                       + HOUSING_09_SCALED \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + RUNSPEED \
                       + NUM_BUS_STOPS \
                       + TRANSBAY \
                       + BART_FROMS \
                       + SHR_HH_0VEH \
                       + SHR_INCOME_100P \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:             LOG_RIDERS   R-squared:                       0.886
Model:                            OLS   Adj. R-squared:                  0.886
Method:                 Least Squares   F-statistic:                 1.100e+04
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:49:07   Log-Likelihood:                -28183.
No. Observations:               16940   AIC:                         5.639e+04
Df Residuals:                   16928   BIC:                         5.648e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
EMP_WAC_SCALED        0.0014   6.78e-0

In [52]:
# RUN
# try different employment categories
mod = smf.formula.ols(formula="LOG_RIDERS \
                       ~ EMP_WAC_SCALED \
                       + RETAIL_WAC_SCALED \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + BART_FROMS \
                       + TRANSBAY \
                       + SHR_HH_0VEH \
                       + SHR_INCOME_100P \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:             LOG_RIDERS   R-squared:                       0.876
Model:                            OLS   Adj. R-squared:                  0.876
Method:                 Least Squares   F-statistic:                     9943.
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:49:07   Log-Likelihood:                -23958.
No. Observations:               14137   AIC:                         4.794e+04
Df Residuals:                   14127   BIC:                         4.801e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
EMP_WAC_SCALED        0.0021   7.63e-0

In [53]:
# RUN
# try different employment categories
mod = smf.formula.ols(formula="LOG_RIDERS \
                       ~ EMP_WAC_SCALED \
                       + RETAIL_WAC_SCALED \
                       + LEISER_WAC_SCALED \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + BART_FROMS \
                       + TRANSBAY \
                       + SHR_HH_0VEH \
                       + SHR_INCOME_100P \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())


                            OLS Regression Results                            
Dep. Variable:             LOG_RIDERS   R-squared:                       0.879
Model:                            OLS   Adj. R-squared:                  0.879
Method:                 Least Squares   F-statistic:                     9347.
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:49:11   Log-Likelihood:                -23749.
No. Observations:               14137   AIC:                         4.752e+04
Df Residuals:                   14126   BIC:                         4.760e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
EMP_WAC_SCALED        0.0014   8.18e-0

In [76]:
#RUN
# try different employment categories
mod = smf.formula.ols(formula="LOG_RIDERS \
                       ~ EMP_WAC_SCALED \
                       + EDHLTH_WAC_SCALED \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + BART_FROMS \
                       + TRANSBAY \
                       + SHR_HH_0VEH \
                       + SHR_INCOME_100P \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())


                            OLS Regression Results                            
Dep. Variable:             LOG_RIDERS   R-squared:                       0.876
Model:                            OLS   Adj. R-squared:                  0.876
Method:                 Least Squares   F-statistic:                 1.192e+04
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        15:15:03   Log-Likelihood:                -28944.
No. Observations:               16940   AIC:                         5.791e+04
Df Residuals:                   16930   BIC:                         5.798e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
EMP_WAC_SCALED        0.0004      0.00

In [55]:
estimate['NONRETAIL_DEN_WAC'] = estimate.EMP_WAC_SCALED - estimate.RETAIL_WAC_SCALED

In [56]:
# RUN
# test retail vs non-retail density
mod = smf.formula.ols(formula="LOG_RIDERS \
                       ~ RETAIL_WAC_SCALED \
                       + NONRETAIL_DEN_WAC \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + BART_FROMS \
                       + SHR_HH_0VEH \
                       + TRANSBAY \
                       + SHR_INCOME_100P \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:             LOG_RIDERS   R-squared:                       0.876
Model:                            OLS   Adj. R-squared:                  0.876
Method:                 Least Squares   F-statistic:                     9943.
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:49:17   Log-Likelihood:                -23958.
No. Observations:               14137   AIC:                         4.794e+04
Df Residuals:                   14127   BIC:                         4.801e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
RETAIL_WAC_SCALED    -0.1014      0.02

In [57]:
estimate.columns

Index(['AGENCY_ID', 'ARRIVAL_TIME_DEV', 'AVG_HH_SIZE', 'BART_FROMS',
       'BART_TOS', 'BIKERACK', 'CALTRAIN_OFF', 'CALTRAIN_ON', 'CAPACITY',
       'CROWDED', 'CROWDHOURS', 'DEPARTURE_TIME_DEV', 'DIR', 'DOORCYCLES',
       'DOW', 'DWELL', 'DWELL_S', 'EDD_EMP', 'EDHLTH_RAC_SCALED',
       'EDHLTH_WAC_SCALED', 'EMP_RAC_SCALED', 'EMP_WAC_SCALED', 'EOL', 'FARE',
       'FREQ_S', 'FULLFARE_REV', 'HEADWAY_S', 'HH_DEN_ACS', 'HU_DEN_ACS',
       'IMP_TRIP_STOPS', 'LOAD_ARR', 'LOAD_DEP', 'MONTH',
       'MUNI_RAIL_ALIGHTINGS', 'MUNI_RAIL_BOARDINGS', 'NUMDAYS',
       'NUM_BUS_STOPS', 'OBSDAYS', 'OBS_TRIP_STOPS', 'OCC_RATE', 'OFF', 'ON',
       'ONTIME5', 'PASSDELAY_ARR', 'PASSDELAY_DEP', 'PASSHOURS', 'PASSMILES',
       'POP_DEN_ACS', 'RDBRDNGS', 'ROUTE_LONG_NAME', 'ROUTE_SHORT_NAME',
       'ROUTE_TYPE', 'RUNSPEED', 'RUNSPEED_S', 'RUNTIME', 'RUNTIME_S', 'SEQ',
       'SERVMILES', 'SERVMILES_S', 'SOL', 'STOPNAME', 'STOPNAME_AVL',
       'STOP_ID', 'TIMEPOINT', 'TOTSPEED', 'TOTSPEED_S', 'TOTTI

In [58]:
estimate['AVG_BART'] = (estimate['BART_FROMS']+estimate['BART_TOS'])/2

In [59]:
for column in estimate.columns:
    print(column)

AGENCY_ID
ARRIVAL_TIME_DEV
AVG_HH_SIZE
BART_FROMS
BART_TOS
BIKERACK
CALTRAIN_OFF
CALTRAIN_ON
CAPACITY
CROWDED
CROWDHOURS
DEPARTURE_TIME_DEV
DIR
DOORCYCLES
DOW
DWELL
DWELL_S
EDD_EMP
EDHLTH_RAC_SCALED
EDHLTH_WAC_SCALED
EMP_RAC_SCALED
EMP_WAC_SCALED
EOL
FARE
FREQ_S
FULLFARE_REV
HEADWAY_S
HH_DEN_ACS
HU_DEN_ACS
IMP_TRIP_STOPS
LOAD_ARR
LOAD_DEP
MONTH
MUNI_RAIL_ALIGHTINGS
MUNI_RAIL_BOARDINGS
NUMDAYS
NUM_BUS_STOPS
OBSDAYS
OBS_TRIP_STOPS
OCC_RATE
OFF
ON
ONTIME5
PASSDELAY_ARR
PASSDELAY_DEP
PASSHOURS
PASSMILES
POP_DEN_ACS
RDBRDNGS
ROUTE_LONG_NAME
ROUTE_SHORT_NAME
ROUTE_TYPE
RUNSPEED
RUNSPEED_S
RUNTIME
RUNTIME_S
SEQ
SERVMILES
SERVMILES_S
SOL
STOPNAME
STOPNAME_AVL
STOP_ID
TIMEPOINT
TOTSPEED
TOTSPEED_S
TOTTIME
TOTTIME_S
TRIP_HEADSIGN
TRIP_STOPS
Total_Housing_Units
Total_Pop
VAC_RATE
VC
WAITHOURS
WGT_TRIP_STOPS
WHEELCHAIR
HOURLY_AVG_ON_PARK
TRANSBAY
HOUSING_09_SCALED
LEISER_RAC_SCALED
LEISER_WAC_SCALED
SHR_HH_0VEH
SHR_INCOME_0_15
SHR_INCOME_100P
SHR_INCOME_15_50
SHR_INCOME_50_100
12_HR_OFF_PUB_PARK
1

In [60]:
# RUN
# test retail vs non-retail density
mod = smf.formula.ols(formula="LOG_RIDERS \
                       ~ RETAIL_WAC_SCALED \
                       + NONRETAIL_DEN_WAC \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + AVG_BART \
                       + SHR_HH_0VEH \
                       + TRANSBAY \
                       + SHR_INCOME_100P \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:             LOG_RIDERS   R-squared:                       0.876
Model:                            OLS   Adj. R-squared:                  0.876
Method:                 Least Squares   F-statistic:                     9944.
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:49:18   Log-Likelihood:                -23957.
No. Observations:               14137   AIC:                         4.793e+04
Df Residuals:                   14127   BIC:                         4.801e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
RETAIL_WAC_SCALED    -0.1012      0.02

In [61]:
for column in estimate.columns.sort_values():
    print(column)

12_HR_OFF_PUB_PARK
1_HR_OFF_PUB_PARK
2_HR_OFF_PUB_PARK
3_HR_OFF_PUB_PARK
4_HR_OFF_PUB_PARK
AGENCY_ID
ARRIVAL_TIME_DEV
AVG_BART
AVG_HH_SIZE
AVG_RIDE
BART_FROMS
BART_TOS
BIKERACK
CALTRAIN_OFF
CALTRAIN_ON
CAPACITY
CROWDED
CROWDHOURS
DAY_DISCOUNT_OFF_PUB_PARK
DAY_OFF_PUB_PARK
DEPARTURE_TIME_DEV
DIR
DOORCYCLES
DOW
DWELL
DWELL_S
EDD_EMP
EDHLTH_RAC_SCALED
EDHLTH_WAC_SCALED
EMP_RAC_SCALED
EMP_WAC_SCALED
EOL
FARE
FREQ_S
FULLFARE_REV
HEADWAY_S
HH_DEN_ACS
HOURLY_AVG_ON_PARK
HOUSING_09_SCALED
HU_DEN_ACS
IMP_TRIP_STOPS
LEISER_RAC_SCALED
LEISER_WAC_SCALED
LOAD_ARR
LOAD_DEP
LOG_RIDERS
MONTH
MUNI_RAIL_ALIGHTINGS
MUNI_RAIL_BOARDINGS
NONRETAIL_DEN_WAC
NUMDAYS
NUM_BUS_STOPS
OBSDAYS
OBS_TRIP_STOPS
OCC_RATE
OFF
ON
ONTIME5
PASSDELAY_ARR
PASSDELAY_DEP
PASSHOURS
PASSMILES
POP_DEN_ACS
RDBRDNGS
RETAIL_RAC_SCALED
RETAIL_WAC_SCALED
ROUTE_LONG_NAME
ROUTE_SHORT_NAME
ROUTE_TYPE
RUNSPEED
RUNSPEED_S
RUNTIME
RUNTIME_S
SEQ
SERVMILES
SERVMILES_S
SHR_HH_0VEH
SHR_INCOME_0_15
SHR_INCOME_100P
SHR_INCOME_15_50
SHR_INCOME_50_1

In [62]:
# RUN
# trying different income categories in the next few
mod = smf.formula.ols(formula="LOG_RIDERS \
                       ~ EMP_WAC_SCALED \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + SHR_HH_0VEH \
                       + SHR_INCOME_15_50 \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())


                            OLS Regression Results                            
Dep. Variable:             LOG_RIDERS   R-squared:                       0.888
Model:                            OLS   Adj. R-squared:                  0.888
Method:                 Least Squares   F-statistic:                     3117.
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:49:19   Log-Likelihood:                -4796.1
No. Observations:                2754   AIC:                             9606.
Df Residuals:                    2747   BIC:                             9648.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
EMP_WAC_SCALED       0.0008      0.000  

In [63]:
# RUN

mod = smf.formula.ols(formula="LOG_RIDERS \
                       ~ EMP_WAC_SCALED \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + SHR_HH_0VEH \
                       + SHR_INCOME_50_100 \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())


                            OLS Regression Results                            
Dep. Variable:             LOG_RIDERS   R-squared:                       0.887
Model:                            OLS   Adj. R-squared:                  0.887
Method:                 Least Squares   F-statistic:                     3075.
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:49:19   Log-Likelihood:                -4812.7
No. Observations:                2754   AIC:                             9639.
Df Residuals:                    2747   BIC:                             9681.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
EMP_WAC_SCALED        0.0006      0.00

In [64]:
# RUN
# this seems to be the preferred model for now...
#the 100P is the easiest to explain but the 50k to 100k has the highest t statistic

mod = smf.formula.ols(formula="LOG_RIDERS \
                       ~ EMP_WAC_SCALED \
                       + ONTIME5 \
                       + FREQ_S \
                       + EOL\
                       + SOL \
                       + SHR_HH_0VEH \
                       + SHR_INCOME_100P \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())
res.save('Model.jpg')

                            OLS Regression Results                            
Dep. Variable:             LOG_RIDERS   R-squared:                       0.872
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                 1.653e+04
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:49:19   Log-Likelihood:                -29164.
No. Observations:               16940   AIC:                         5.834e+04
Df Residuals:                   16933   BIC:                         5.840e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
EMP_WAC_SCALED      0.0016    6.8e-05     

In [69]:
columns = ['EMP_WAC_SCALED', 'ONTIME5', 'FREQ_S', 'SHR_HH_0VEH', 'SHR_INCOME_100P']


In [70]:
for column in columns:
    estimate[column + '_LOG'] = np.log(estimate[column] + 1)

In [68]:
# RUN
# this seems to be the preferred model for now...
#the 100P is the easiest to explain but the 50k to 100k has the highest t statistic

mod = smf.formula.ols(formula="LOG_RIDERS \
                       ~ EMP_WAC_SCALED_LOG \
                       + ONTIME5_LOG \
                       + FREQ_S_LOG \
                       + EOL \
                       + SOL \
                       + SHR_HH_0VEH_LOG \
                       + SHR_INCOME_100P_LOG \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:             LOG_RIDERS   R-squared:                       0.885
Model:                            OLS   Adj. R-squared:                  0.885
Method:                 Least Squares   F-statistic:                 1.859e+04
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:50:00   Log-Likelihood:                -28287.
No. Observations:               16940   AIC:                         5.659e+04
Df Residuals:                   16933   BIC:                         5.664e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
EMP_WAC_SCALED_LOG      0.0825    

In [71]:
estimate['CALTRAIN_AVG'] = (estimate.CALTRAIN_OFF + estimate.CALTRAIN_ON)/2
estimate['MUNI_RAIL_AVG'] = (estimate.MUNI_RAIL_ALIGHTINGS + estimate.MUNI_RAIL_BOARDINGS)/2

In [72]:
columns = ['AVG_BART', 'CALTRAIN_AVG', 'HOURLY_AVG_ON_PARK', 'MUNI_RAIL_AVG']
for column in columns:
    estimate[column + '_LOG'] = np.log(estimate[column]+1)

In [73]:
# RUN
# drop muni rail and caltrain because they have high p values

mod = smf.formula.ols(formula="LOG_RIDERS \
                       ~ EMP_WAC_SCALED_LOG \
                       + FREQ_S_LOG \
                       + EOL_LOG \
                       + SOL_LOG \
                       + SHR_INCOME_100P_LOG \
                       + TRANSBAY \
                       + HOURLY_AVG_ON_PARK_LOG \
                       + AVG_BART \
                       -1",
             data = estimate)

res = mod.fit()
print(res.summary())
res.save('Model.jpg')

                            OLS Regression Results                            
Dep. Variable:             LOG_RIDERS   R-squared:                       0.887
Model:                            OLS   Adj. R-squared:                  0.887
Method:                 Least Squares   F-statistic:                 1.667e+04
Date:                Fri, 14 Jul 2017   Prob (F-statistic):               0.00
Time:                        11:51:04   Log-Likelihood:                -28102.
No. Observations:               16938   AIC:                         5.622e+04
Df Residuals:                   16930   BIC:                         5.628e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
EMP_WAC_SCALED_LOG         0